<a href="https://colab.research.google.com/github/LeeSeungwon89/Lecture-and-Self-study/blob/master/CHAPTER4%20%EB%8D%B0%EC%9D%B4%ED%84%B0%20%ED%91%9C%ED%98%84%EA%B3%BC%20%ED%8A%B9%EC%84%B1%20%EA%B3%B5%ED%95%99.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4.1 범주형 변수

## 4.1.1 원-핫-인코딩(가변수)

### 범주형 데이터 문자열 확인하기

## 4.1.2 숫자로 표현된 범주형 특성

# 4.2 OneHotEncoder와 ColumnTransformer: scikit-learn으로 범주형 변수 다루기

# 4.3 make_column_transformer로 간편하게 ColumnTransformer 만들기

# 4,4 구간 분할, 이산화 그리고 선형 모델, 트리 모델

# 4.5 상호작용과 다항식

# 4.6 일변량 비선형 변환

# 4.7 특성 자동 선택

## 4.7.1 일변량 통계

## 4.7.2 모델 기반 특성 선택

## 4.7.3 반복적 특성 선택

# 4.8 전문가 지식 활용

# 4.9 요약 및 정리